In [149]:
from ExtractTable import ExtractTable
import os
import random
import ast
from Levenshtein import distance as lev
import pytesseract
from pdf2image import convert_from_path
import pandas as pd
import logging
import sys
import pytesseract
import re

logging.getLogger("requests").setLevel(logging.WARNING)
logging.getLogger("urllib3").setLevel(logging.WARNING)

In [150]:
# Logger
logging.basicConfig(stream=sys.stdout,
                    level=logging.DEBUG,
                    format='%(asctime)s %(name)-12s %(levelname)-8s %(lineno)d %(message)s')
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

### 0 - Mapping labels

In [151]:
# mapping: key: labels qu'on souhaite retourner, values: toutes les typo qu'on retrouve dans les documents
# final_inversed_mapping: plus utile pour faire une for loop lorsqu'on effectuera le mapping

mapping = {
	
		"reference": '["Code13Réf", "Code Produit", "CIP", "Code CIP", "Code", \
        "Gamme GAVISCONELL Désignation (CIP)", "Code produit", "Code produit (ou Code LPP)	", \
    "Code Article", "C.I.P / Article", "Code EAN"]',
		"quantite": '["Qté", "Qté Vendue", "Nb unité", "Qt", "Qte Facturée", "Qte Délivrée", "quantité", \
    "Quantité", "Qt é", "Nombre Unités", "Nombre", "Quantité Lieu", "Cdée", "Qt e", "Unités"]',
		"CA TTC": '["Montant TTC", "CA Brut TTC", "CA TTC brut", "PV TTC", "CA brut TTC", "PrixV TTC", \
    "Ch Affaire T.T.C.", "CA TTC", ]',
		"CA HT": '["Rémunération HT", "Chiffre Affraire HT", "Chiffre", "Ch Affaire H.T.", "CA HT", \
    "Montant cdé", "Valeur", "Montant HT", ]',
        "CA": '["CA"]',
		"prix unitaire HT": '["Prix Vente", "Prix Unitaire €", "Px vente", \
    "PU HT", "HT U Brut", "Prix HT unitaire", "Prix unitaire HT", ]',
    "prix unitaire TTC": '["Prix TTC", "Prix Vente TTC", "PVTTC REMISE"]',
          }

inversed_mapping = {value: key for key, value in mapping.items()}

final_inversed_mapping = {}
for key, value in inversed_mapping.items():
    key = ast.literal_eval(key)
    if type(key) == list:
        for item in key:
            final_inversed_mapping[item] = value
    else:
        final_inversed_mapping[key] = value

# all_possible_elements is a list with all possible labels discovered in the docs
all_possible_elements = [ast.literal_eval(l) for l in mapping.values()]
all_possible_elements = [item for sublist in all_possible_elements for item in sublist]

# ExtractTable API
Cet API Python permet d'extraire magnifiquement bien un tableau d'un document PDF
Raphael Zeitoun a pris 5.000 crédits

In [152]:
api_key = "sJCGEoGgykvttxEq8bN6XJTzjWukOVIeL0jULE8p"
et_sess = ExtractTable(api_key)
print(et_sess.check_usage())        # Checks the API Key validity as well as shows associated plan usage 

{'credits': 5000, 'queued': 0, 'used': 632}


### 1 - Functions

In [153]:
def sanity_checks_after_API_and_cleaning(dict_outputs, filename_pdf):
    """
    Input: Dict output after cleaning
    Output: new dict with sanity check
    """
    # Need to use the df after cleaning name
    df_outputs = pd.DataFrame(dict_outputs)
    
    # 1 - Check that reference number contains 13 digits
    if "reference" in df_outputs.columns.tolist():
        mask = (df_outputs['reference'].str.replace(' ', '').str.len() == 13)
        df_outputs = df_outputs.loc[mask]
    else:
        logger.info("reference not detected")
        reference_pytesseract = pytesseract_for_ref(filename_pdf)
        df_outputs["reference"] = [reference_pytesseract for i in range(len(df_outputs))]
    
    return df_outputs


def sanity_check_before_API(file):
    """
    Input: file name
    Output: elements that the API will missed
    """
    # 3 - Objectif: Si reference manque:
    reference = fix_special_cases(file)
    return reference


def sanity_check_just_after_API(df_outputs):
    # Need to use the df before cleaning name
    
    # 1 - Objectif: choisir les bonnes colonnes
    # parfois certains noms de colonnes sont des duplicates ... exemple (proofs-picture_proof-6962_la pharmatheque ratarieux-listing_listerine.pdf)
    duplicates_columns = list(set([col for col in df_outputs.columns.tolist() if df_outputs.columns.tolist().count(col) > 1]))
    if len(duplicates_columns) > 0:
        logger.info("Duplicates columns detected")
        df_outputs = df_outputs.iloc[:, 0:4] # on garde les 3 premieres colonnes. Cette technique est hardcoée sur le document exemple
       
    return df_outputs, df_outputs.to_dict()


def fix_special_cases(file):
    # 1 - Objectif: reperer la reference du produit
    # Reference produit en titre (exemple: proofs-picture_proof-9439_PHARMACIE DE L ELIXIR-eludril.pdf)
    # TODO: comment detecter qu'il s'agit de ce type de document ? Appliquer tesseract en premier ?
    # commencer par ExtractTable, voir qu'il manque la ref, puis passer a tesseract et reutiliser ExtractTable ?
    # Utilisons le nom du doc pour l'instant:
    if 'PHARMACIE DE L ELIXIR' in file:
        logger.info('PHARMACIE DE L ELIXIR special case')
        text_image = from_pdf_to_jpg(file)
        l = [element for element in text_image if element != ' ']
        ref = ''
        for element in l:
            if element.isdigit():
                ref += element
            else:
                ref = ''
            if len(ref) == 13:
                break
        return ref
    else:
        return None
     
                              
def pytesseract_for_ref(filename_pdf):
    pages = convert_from_path(filename_pdf)
    for i in range(len(pages)):
        file = '{}_page_{}.jpg'.format(filename_pdf[:-4], i)
        pages[i].save(file, 'JPEG')
    # On lit seulement la page 0 (on ne prend pas en compte les cas ou les pdf > 1 page)
    file = '{}_page_{}.jpg'.format(filename_pdf[:-4], 0)
    text = pytesseract.image_to_string(file)
    # Expression régulière pour les séquences de 13 chiffres
    regex = r"\d{13}"
    matches = re.findall(regex, s.replace(" ", ""))[0]

    return matches

In [154]:
def extract_table(path_image):
    """
    Input: document type A file path (pdf)
    Output: print df of retrieved info
    PS:To process PDF, make use of pages ("1", "1,3-4", "all") params in the read_pdf function


    """
    #table_data = et_sess.process_file(filepath=path_image, output_format="df")
    table_data = et_sess.process_file(filepath=path_image, output_format="df", pages="all")
    et_sess.check_usage()
    print(table_data)
    
    
def get_unique_number_pharma(filename):
    """
    Il existe deux types de filenames dans les listing de pharma recus:
    type 1: contient le mot deal
    type 2: contient le mot proof
    En fonction du type, on split d'une certaine façon et réucpère le numéro de la pharmacie, 
    afin d'appliquer ExtractTable sur un seul numéro à chaque fois, et eviter 100 applications de l'API
    si nous avons 100 listings de la même pharma
    """
    if "deal" in filename:
        number = filename.split("_")[1].split("-")[0]
    else:
        number = filename.split("-")[2].split("_")[0]
    return number


def update_retrieved_dictionnary(d, final_inversed_mapping):
    """
    Input:
    - d: dictionnary retireved by ExtractTable API (via retrieve_info() function)
    - final_inversed_mapping: mapping dictionnary which we inversed so that we can do the mapping
    Output:
    cleaned dict
    """

    keys2rename = []
    values2add = []
    keysNotFound = []
    
    # 1. Mapping
    for key, value in d.items():
        if key in list(final_inversed_mapping.keys()):
            keys2rename.append(key)
            values2add.append(value)
        else:
            keysNotFound.append(key)
    
    # 2. For renaming the keys with the correct ones, we need to remove the old ones and add the new ones
    # 2.1 Remove old
    for key in keys2rename:
        del d[key]
    
    # 2.2 Add new
    for key, value in zip(keys2rename, values2add):
        d[final_inversed_mapping[key]] = value
    
    # 3. Remove the non mapped key (maybe we did not succeeded to match - Tests word similarity)
    for key in keysNotFound:
        del d[key]
    return d


def retrieve_info(path_image, pp):
    """
    Input: document type A file path (pdf)
    pp: if True, print all the extractions
    Output: dictionnary with all info
    PS:To process PDF, make use of pages ("1", "1,3-4", "all") params in the read_pdf function


    """
    #table_data = et_sess.process_file(filepath=path_image, output_format="df")
    table_data = et_sess.process_file(filepath=path_image, output_format="df", pages="all")
    logger.info(et_sess.check_usage())
    if pp:
        print(table_data)
    df = table_data[0]
    df.columns = df.loc[0] # maybe can vary from a doc to another
    #df = df[2:] # maybe can vary from a doc to another
    return df, df[1:].to_dict(), table_data


def pipeline(filename, pp=False, api=True, df_info=False):
    """
    Pipeline
    Input: Filename
    pp: if True, print all the extractions
    api: if True, we apply ExtractTable API. If False, it means that we already applied it and want
    to apply the function on the extraced df
    df_info: exists only if we already applied the function with api = True the first time
    Output: df & dictionnary with the relevant informations (ExtractTable API + mapping)"""
    # Check few things before applying API
    logger.info("Filename: {} \n".format(filename))
    reference = sanity_check_before_API(file)
    
    if api:
        df_info, dict_info, table_data = retrieve_info("../Data/PDF/{}".format(filename), pp)
    else:
        table_data = None
    df_info, dict_info = sanity_check_just_after_API(df_info)
    print(df_info.head())
    
    # On a souvent des pb de colonnes, c'est a dire que parfois, les bons noms de colonnes se situent dans les
    # lignes, dûs à des noms de colonnes trop longs repérés par l'API. On utilise une while loop dans laquelle
    # on remonte les lignes petit à petit jusqu'à ce que le code reconnaisse des noms labels
    final_dict = {}
    # Tant que le final_dict (output) est vide ou bien tant que la df output n'est pas 1 (ce qui signifie
    # qu'il reste encore des lignes à remonter pour les faire devenir columns name, on continue)
    i = 1
    while len(final_dict) == 0 and len(df_info) != 1:
        logger.info("Apply function update_retrieved_dictionnary: time {}".format(i))
        final_dict = update_retrieved_dictionnary(dict_info, final_inversed_mapping)
        df_info.columns = df_info.loc[0]
        df_info = df_info[1:]
        dict_info = df_info.to_dict()
        df_info.reset_index(drop=True, inplace=True)
        i+=1
        
    if reference:
        logger.info("reference was spotted outside of the df")
        # TODO verifier sur un exemple si la ligne ce dessous fonctionne
        final_dict["reference"] = {i: reference for i in range(len(final_dict.keys()))[0]}
    
    # Check few things after applying API
    final_dict = sanity_checks_after_API_and_cleaning(final_dict, "../Data/PDF/{}".format(filename))
    return df_info, final_dict, table_data
    

def from_pdf_to_jpg(file):
    """Input: pdf file
    Ouput: Jpg file, usign tesseract"""
    logger.info(file)
    pages = convert_from_path("../Data/PDF/{}".format(file))
    for i in range(len(pages)):
        pages[i].save('../Data/JPEG/' + file[:-4]+ str(i) +'.jpg', 'JPEG')
    # Tesseract
    path_image = "../Data/JPEG/{}".format(file[:-4]+ str(i) +'.jpg')
    text_image = pytesseract.image_to_string(path_image)   

In [309]:
path_image = "Data/Exemples/typeB.pdf"
#document_A(path_image)

In [73]:
#df_info, dict_info, table_data = retrieve_info("../Data/PDF/{}".format(file), False)

2023-03-07 17:53:21,751 urllib3.connectionpool DEBUG    452 https://trigger.extracttable.com:443 "POST / HTTP/1.1" 200 1310
2023-03-07 17:53:22,286 urllib3.connectionpool DEBUG    452 https://validator.extracttable.com:443 "GET / HTTP/1.1" 200 97
2023-03-07 17:53:22,289 root         INFO     77 {'credits': 5000, 'queued': 0, 'used': 606}


### 3 - Model
This is a first baseline model. Lets test it

In [6]:
filenames = []
for filename in os.listdir("../Data/PDF"):
    filenames.append(filename)
print("Total elemets in 'filenames' list: {}".format(len(filenames)))

Total elemets in 'filenames' list: 2744


In [156]:
# Test the model on some examples
file = "proofs-picture_proof-7699_pharmacie du marché 92250-gallia.pdf"
#file = filenames[12]

df, dic, table_data = pipeline(file, pp=False, api=True, df_info=False)

next_table = 1

while len(dic) == 0:
    logger.info("Nothing deteted. Lets test on next table: table {}".format(next_table))
    logger.info("{} tables detected".format(len(table_data)))
    df, dic, _ = pipeline(filename, pp=False, api=False, df_info=table_data[next_table])
    next_table += 1

2023-03-08 11:52:03,738 root         INFO     97 Filename: proofs-picture_proof-7699_pharmacie du marché 92250-gallia.pdf 



/Users/remyadda/opt/anaconda3/lib/python3.8/site-packages/ExtractTable/parsers.py:43: UserWarning: Check the status using the JobId after 10 seconds
  warnings.warn(_msg_)


[Info]: Waiting to retrieve the output; JobId: 9d17710cf534edee8b552995915df165581f8eb4d26006401d178986fa28b800
2023-03-08 11:52:15,620 root         INFO     78 {'credits': 5000, 'queued': 0, 'used': 636}
0                                        CIP LibellO  Janvier  DOcembre  \
0                                        CIP LibellO  Janvier  DOcembre   
1  3041091124128 BLEDINER FAR INST LEG POTAGER B/...        0         0   
2                 3041091224064 BLEDINER RIZ CAROTTE        0         0   
3  3041091474711 GALLIA BB EXP AC TRANSIT 1 LAIT ...        0         1   
4  3041091474988 GALLIA BB EXP AC TRANSIT 2 LAIT ...        2         6   

0  Novembre  
0  Novembre  
1         0  
2         0  
3         6  
4         5  
2023-03-08 11:52:15,627 root         INFO     115 Apply function update_retrieved_dictionnary: time 1
2023-03-08 11:52:15,630 root         INFO     115 Apply function update_retrieved_dictionnary: time 2
2023-03-08 11:52:15,633 root         INFO     115 Apply 

<ipython-input-154-3088dd81e58f>:119: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  dict_info = df_info.to_dict()


2023-03-08 11:52:19,050 root         INFO     10 Nothing deteted. Lets test on next table: table 1
2023-03-08 11:52:19,052 root         INFO     11 3 tables detected
2023-03-08 11:52:19,052 root         INFO     97 Filename: proofs-picture_proof-9160_GRANDE PHARMACIE DE LA CROIX ROUGE-Listing_sortie_Eluday_Plan_Trade_MArs_2022.pdf 

  0 1 2 3 4 5         6 7  8   9 10  11     12  13
0                                    PV  TTC 2  97
1              PRODUITS        3     13      3  42
2                                    64     21  22
3                          0   5     39     20  55
4                          1  22     85     22  32
2023-03-08 11:52:19,060 root         INFO     115 Apply function update_retrieved_dictionnary: time 1
2023-03-08 11:52:19,063 root         INFO     115 Apply function update_retrieved_dictionnary: time 2
2023-03-08 11:52:19,066 root         INFO     115 Apply function update_retrieved_dictionnary: time 3
2023-03-08 11:52:19,070 root         INFO     115 App

<ipython-input-154-3088dd81e58f>:119: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  dict_info = df_info.to_dict()


2023-03-08 11:52:22,497 root         INFO     10 Nothing deteted. Lets test on next table: table 2
2023-03-08 11:52:22,499 root         INFO     11 3 tables detected
2023-03-08 11:52:22,500 root         INFO     97 Filename: proofs-picture_proof-9160_GRANDE PHARMACIE DE LA CROIX ROUGE-Listing_sortie_Eluday_Plan_Trade_MArs_2022.pdf 

    0   1
0   4  90
1   3  90
2  18  90
3  17  60
4  19  20
2023-03-08 11:52:22,504 root         INFO     115 Apply function update_retrieved_dictionnary: time 1
2023-03-08 11:52:22,507 root         INFO     115 Apply function update_retrieved_dictionnary: time 2
2023-03-08 11:52:22,509 root         INFO     115 Apply function update_retrieved_dictionnary: time 3
2023-03-08 11:52:22,511 root         INFO     115 Apply function update_retrieved_dictionnary: time 4
2023-03-08 11:52:22,513 root         INFO     115 Apply function update_retrieved_dictionnary: time 5
2023-03-08 11:52:22,515 root         INFO     115 Apply function update_retrieved_dictionnary: 

IndexError: list index out of range

In [ ]:
df.head(3)

In [ ]:
dic